# Add Embeddings to Table

<div style="display: inline-flex; align-items: center; gap: 10px;">
        <a href="https://colab.research.google.com/github/3lc-ai/3lc-examples/blob/main/tutorials/bb-embeddings/3-add-embeddings-to-table.ipynb"
        target="_blank"
            style="background-color: transparent; text-decoration: none; display: inline-flex; align-items: center;
            padding: 5px 10px; font-family: Arial, sans-serif;"> <img
            src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="height: 30px;
            vertical-align: middle;box-shadow: none;"/>
        </a> <a href="https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/bb-embeddings/3-add-embeddings-to-table.ipynb"
            style="text-decoration: none; display: inline-flex; align-items: center; background-color: #ffffff; border:
            1px solid #d1d5da; border-radius: 8px; padding: 2px 10px; color: #333; font-family: Arial, sans-serif;">
            <svg aria-hidden="true" focusable="false" role="img" class="octicon octicon-mark-github" viewBox="0 0 16 16"
            width="20" height="20" fill="#333"
            style="display:inline-block;user-select:none;vertical-align:text-bottom;overflow:visible; margin-right:
            8px;">
                <path d="M8 0c4.42 0 8 3.58 8 8a8.013 8.013 0 0 1-5.45 7.59c-.4.08-.55-.17-.55-.38 0-.27.01-1.13.01-2.2
                0-.75-.25-1.23-.54-1.48 1.78-.2 3.65-.88 3.65-3.95 0-.88-.31-1.59-.82-2.15.08-.2.36-1.02-.08-2.12 0
                0-.67-.22-2.2.82-.64-.18-1.32-.27-2-.27-.68 0-1.36.09-2 .27-1.53-1.03-2.2-.82-2.2-.82-.44 1.1-.16
                1.92-.08 2.12-.51.56-.82 1.28-.82 2.15 0 3.06 1.86 3.75 3.64 3.95-.23.2-.44.55-.51
                1.07-.46.21-1.61.55-2.33-.66-.15-.24-.6-.83-1.23-.82-.67.01-.27.38.01.53.34.19.73.9.82 1.13.16.45.68
                1.31 2.69.94 0 .67.01 1.3.01 1.49 0 .21-.15.45-.55.38A7.995 7.995 0 0 1 0 8c0-4.42 3.58-8 8-8Z"></path>
            </svg> <span style="vertical-align: middle; color: #333;">Open in GitHub</span>
        </a>
</div>

In this tutorial, we will take a set of previously computed embeddings and associate them with their corresponding bounding box in a Table representing
a COCO-style object-detection dataset.

This will create a Table with an additional column containing the embeddings for each bounding box.

To run this notebook, you must also have run:
* [1-fine-tune-on-crops.ipynb](https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/bb-embeddings/1-fine-tune-on-crops.ipynb)
* [2-collect-embeddings.ipynb](https://github.com/3lc-ai/3lc-examples/blob/main/tutorials/bb-embeddings/2-collect_embeddings.ipynb)

## Project Setup

In [ ]:
PROJECT_NAME = "Bounding Box Classifier"
TRANSIENT_DATA_PATH = "../../transient_data"
BATCH_SIZE = 32
DATASET_NAME = "Balloons"

## Imports

In [ ]:
from copy import deepcopy

import numpy as np
import tlc
from tqdm.auto import tqdm

## Get Original Table

In [ ]:
# Open the Table created in the first notebook
original_table = tlc.Table.from_names(
    project_name=PROJECT_NAME,
    dataset_name=DATASET_NAME,
    table_name="original"
)

## Create New Table with Embeddings

In [ ]:
# Open the embeddings created in previous notebook
embeddings = np.load(TRANSIENT_DATA_PATH + "/bb_classifier_embeddings.npy")

# Create schema for embedding
embedding_schema = tlc.FloatVector3Schema()

# Create a schema for the new table
new_table_schema = deepcopy(original_table.rows_schema)
new_table_schema.values["bbs"].values["bb_list"].add_sub_schema("embedding", embedding_schema)

In [ ]:
# Create a TableWriter for the new table
table_writer = tlc.TableWriter(
    project_name=PROJECT_NAME,
    dataset_name=DATASET_NAME,
    table_name="embeddings_added",
    column_schemas=new_table_schema.values,
    if_exists="overwrite"
)

# Iterate over the rows of the original table and add the embeddings
i = 0
for row in tqdm(original_table):
    new_row = deepcopy(row)
    for bb in new_row["bbs"]["bb_list"]:
        bb["embedding"] = embeddings[i].tolist()
        i += 1

    table_writer.add_row(new_row)

# Create the new table
table_writer.finalize()
    